# Yoruba TTS Training on Google Colab

This notebook sets up and trains a VITS-based Text-to-Speech model for Yoruba language.

**Features:**
- GPU-accelerated training
- Automatic setup and dependency installation
- Training progress monitoring
- Audio synthesis testing

**Note:** Make sure to enable GPU runtime: `Runtime > Change runtime type > GPU`

## 1. Setup Environment

In [1]:
# Check GPU availability
!nvidia-smi

Sat Nov 29 16:26:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
# Clone the repository
!git clone https://github.com/T-ultrafast/Naija_tts.git
%cd Naija_tts

Cloning into 'Naija_tts'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 28 (delta 5), reused 28 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 120.69 KiB | 13.41 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/Naija_tts


In [10]:
# Install dependencies
!pip install --upgrade pip
!pip install -q TTS==0.22.0 torch tirchaudio librosa accelerate einops
!pip install -q flask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: Ignored the following versions that require a different python version: 0.0.10.2 Requires-Python >=3.6.0,<3.9; 0.0.10.3 Requires-Python >=3.6.0,<3.9; 0.0.11 Requires-Python >=3.6.0,<3.9; 0.0.12 Requires-Python >=3.6.0,<3.9; 0.0.13.1 Requires-Python >=3.6.0,<3.9; 0.0.13.2 Requires-Python >=3.6.0,<3.9; 0.0.14.1 Requires-Python >=3.6.0,<3.9; 0.0.15 Requires-Python >=3.6.0,<3.9; 0.0.15.1 Requires-Python >=3.6.0,<3.9; 0.0.9 Requires-Python >=3.6.0,<3.9; 0.0.9.1 Requires-Python >=3.6.0,<3.9; 0.0.9.2 Requires-Python >=3.6.0,<3.9; 0.0.9a10 Requires-Python >=3.6.0,<3.9; 0.0.9a9 Requires-Python >=3.6.0,<3.9; 0.1.0 Requires-Python >=3.6.0,<3.10; 0.1.1 Requires-Python >=3.6.0,<3.10; 0.1.2 Requires-Python >=3.6.0,<3.10; 0.1.3 Requires-Python >=3.6.0,<3.10; 0.10.0 Requires-Pyt

## 2. Verify Setup

In [4]:
# List files
!ls -la

total 16
drwxr-xr-x 1 root root 4096 Nov 29 16:26 .
drwxr-xr-x 1 root root 4096 Nov 29 16:25 ..
drwxr-xr-x 4 root root 4096 Nov 20 14:30 .config
drwxr-xr-x 1 root root 4096 Nov 20 14:30 sample_data


In [5]:
# Check metadata file
!head -5 metadata_yor.csv

head: cannot open 'metadata_yor.csv' for reading: No such file or directory


## 3. Update Config for GPU Training

In [6]:
# Update train_vits.py to use CUDA if available
import os

# Read the current train_vits.py
with open('train_vits.py', 'r') as f:
    content = f.read()

# Replace use_cuda=False with use_cuda=True in the script if needed
# (The Trainer should auto-detect GPU, but we can verify)
print("train_vits.py is ready for training")
print(f"CUDA available: {__import__('torch').cuda.is_available()}")

FileNotFoundError: [Errno 2] No such file or directory: 'train_vits.py'

## 4. Start Training

**Note:** Training will take several hours. You can monitor progress in the output below.

In [ ]:
# Start training (this will run for a long time)
!python train_vits.py

## 5. Monitor Training (Optional)

You can check training logs and checkpoints while training is running.

In [ ]:
# List output directories
!ls -lh out/naija_xtts_yor/

In [ ]:
# View latest training log (update the directory name to match your run)
!tail -50 out/naija_xtts_yor/*/trainer_0_log.txt

## 6. Test Inference

After training (or using an existing checkpoint), test the model.

In [ ]:
# Find the latest checkpoint
import glob
import os

checkpoint_dirs = glob.glob('out/naija_xtts_yor/*/')
if checkpoint_dirs:
    latest_dir = max(checkpoint_dirs, key=os.path.getmtime)
    checkpoints = glob.glob(os.path.join(latest_dir, 'checkpoint_*.pth'))
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getmtime)
        print(f"Latest checkpoint: {latest_checkpoint}")
    else:
        print("No checkpoints found yet")
else:
    print("No training runs found")

In [ ]:
# Test synthesis
from TTS.utils.synthesizer import Synthesizer
from naija_formatter import naija_formatter
import TTS.tts.datasets
from IPython.display import Audio

# Register formatter
TTS.tts.datasets.naija = naija_formatter

# Update these paths to match your latest checkpoint
MODEL_PATH = latest_checkpoint  # Use the checkpoint found above
CONFIG_PATH = os.path.join(os.path.dirname(latest_checkpoint), 'config.json')

# Load model
synthesizer = Synthesizer(
    tts_checkpoint=MODEL_PATH,
    tts_config_path=CONFIG_PATH,
    use_cuda=True,  # Use GPU for inference
)

# Synthesize
text = "Bawo ni, se dada ni?"
print(f"Synthesizing: {text}")
wav = synthesizer.tts(text)

# Play audio
Audio(wav, rate=synthesizer.output_sample_rate)

## 7. Save Checkpoint to Google Drive (Optional)

To preserve your trained model, save it to Google Drive.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Copy checkpoints to Drive
!mkdir -p /content/drive/MyDrive/yoruba_tts_checkpoints
!cp -r out/naija_xtts_yor/* /content/drive/MyDrive/yoruba_tts_checkpoints/
print("Checkpoints saved to Google Drive!")

## 8. Run Web Interface (Optional)

You can run the Flask web interface in Colab using ngrok for public access.

In [ ]:
# Install pyngrok
!pip install -q pyngrok

In [ ]:
# Update app.py to use the latest checkpoint
# Then start the Flask app with ngrok
from pyngrok import ngrok
import threading

# Start Flask in background
def run_flask():
    os.system('python app.py')

thread = threading.Thread(target=run_flask)
thread.start()

# Create ngrok tunnel
public_url = ngrok.connect(5000)
print(f"\n🌐 Web Interface URL: {public_url}")
print("Click the link above to access your Yoruba TTS web interface!")